# Cartographie des diffusions
Afin de pouvoir étudier les relartions sémentiques entre les diffusions, on se donne pour but de les visualiser sur le plan.
Pour cela, on commence par en génerer un embedding avec un LLM comme *BERT. Cela nous donne une qualification sémantique de la diffusion.
On réalise une PCA vers une dimension 50 pour alléger la charge que l'on va mettre sur la réduction de dimension t-SNE que l'on opère enfin pour projeter les diffusions dans le plan.

In [1]:
import pathlib as p
import json
import pandas as pd

In [2]:
df = pd.read_csv(
    p.Path("../dataset") / "transcripts" / "whisper.csv",
    index_col="magnetothequeId",
    nrows=300
)

In [3]:
def extract_transcript(json_string: str) -> str:
    return "".join(s["text"] for s in json.loads(json_string))

In [4]:
documents : list[str] = [
    "".join(s["text"] for s in json.loads(ws))
    for ws in df.segments.dropna().tolist()
]

from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("etalab-ia/camembert-base-squadFR-fquad-piaf")
model = AutoModelForQuestionAnswering.from_pretrained("etalab-ia/camembert-base-squadFR-fquad-piaf", output_attentions=True)

ModuleNotFoundError: No module named 'transformers'

Les _attentions_ sont les valeurs d'activation intermediaires de la derniere couche du réseau de neurones dans la variable `model` lorsque l'on lui feed un contenu. Les embeddings d'un texte sont les attentions de `model(transcript)`. Pour finir ce notebook, il faut :
  - calculer les attentions de tous les textes a traiter,
  - calculer une PCA de ces attentions (avec un objet `sklearn.decomposition.PCA(n_components=50)` puis `fit_transform(attentions)` par exemple)
  - calculer une t-SNE un peu comme pour la PCA,
  - plot les points sur un graphique `matplotlib`
  - colorer par `stationId` ou bien avec un k-Means.